In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import lightgbm
import xgboost as xgb

import seaborn as sns
sns.set_style("darkgrid")

C:\Users\Helia\anaconda3\lib\site-packages\cupy\_environment.py:211: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'


In [2]:
#Read the training data from the provided file and save it in the variable dataset as a DataFrame object
training_data = pd.read_csv("data/processed_training_data.csv")
training_data

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool,score
0,0,1,2013-04-04 08:32:15,12,187,219,893,3,3.5,1,...,1,0,4,0,1,1,1,0,0,0
1,1,1,2013-04-04 08:32:15,12,187,219,10404,4,4.0,1,...,1,0,4,0,1,1,1,0,0,0
2,2,1,2013-04-04 08:32:15,12,187,219,21315,3,4.5,1,...,1,0,4,0,1,1,1,0,0,0
3,3,1,2013-04-04 08:32:15,12,187,219,27348,2,4.0,1,...,1,0,4,0,1,1,1,0,0,0
4,4,1,2013-04-04 08:32:15,12,187,219,29604,4,3.5,1,...,1,0,4,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3470838,3470838,233039,2013-05-19 11:53:45,5,219,219,52908,3,4.0,1,...,4,18,2,1,1,1,0,0,0,0
3470839,3470839,233039,2013-05-19 11:53:45,5,219,219,61253,2,3.0,1,...,4,18,2,1,1,1,0,0,0,0
3470840,3470840,233039,2013-05-19 11:53:45,5,219,219,66013,2,3.0,0,...,4,18,2,1,1,1,0,0,0,0
3470841,3470841,233039,2013-05-19 11:53:45,5,219,219,71380,3,4.0,1,...,4,18,2,1,1,1,0,0,0,0


In [3]:
#Read the test data from the provided file and save it in the variable dataset as a DataFrame object
validation_data = pd.read_csv("data/processed_validation_data.csv")
validation_data

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool,score
0,3470843,233039,2013-05-19 11:53:45,5,219,219,77380,3,4.5,1,...,4,18,2,1,1,1,0,0,0,0
1,3470844,233039,2013-05-19 11:53:45,5,219,219,80151,3,4.5,1,...,4,18,2,1,1,1,0,0,0,0
2,3470845,233039,2013-05-19 11:53:45,5,219,219,83874,2,3.5,1,...,4,18,2,1,1,1,0,0,0,0
3,3470846,233039,2013-05-19 11:53:45,5,219,219,88579,3,5.0,1,...,4,18,2,1,1,1,0,0,0,0
4,3470847,233039,2013-05-19 11:53:45,5,219,219,93036,3,4.5,1,...,4,18,2,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487499,4958342,332785,2013-06-30 19:55:18,5,219,219,77700,3,4.0,1,...,1,21,3,0,1,0,0,0,0,0
1487500,4958343,332785,2013-06-30 19:55:18,5,219,219,88083,3,4.0,1,...,1,21,3,0,1,0,0,0,0,0
1487501,4958344,332785,2013-06-30 19:55:18,5,219,219,94508,3,3.5,1,...,1,21,3,0,1,0,0,0,0,0
1487502,4958345,332785,2013-06-30 19:55:18,5,219,219,128360,3,5.0,1,...,1,21,3,0,1,0,0,1,1,5


Calculate the accuracy of the model

In [4]:
def calculate_dcg(scores):
    return np.sum(scores / np.log2(np.arange(len(scores)) + 2))

In [5]:
def calculate_ndcg(data):
    score = data["score"]
    ordered_score = sorted(score, reverse=True)
    dcg = calculate_dcg(score)
    idcg = calculate_dcg(ordered_score)
    return dcg/idcg

Test all the benchmarks and see which one of them delivers the best accuracy

In [6]:
def prepare_data(data, attr_rem=["position", "date_time", "score", "booking_bool", "click_bool"]):
    x = data.drop(attr_rem, axis=1)
    y = data["score"]
    groups = data["srch_id"].value_counts(sort=False).sort_index()
    return x, y, groups

In [7]:
x_train, y_train, train_groups = prepare_data(training_data)

In [8]:
xgb_methods = ['gbtree', ]

model = xgb.XGBRanker(  
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9,  
    max_depth=7, 
    n_estimators=500, 
    subsample=0.75 
    )

model.fit(x_train, y_train, group=train_groups, verbose=True)

XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
          importance_type='gain', interaction_constraints='', learning_rate=0.1,
          max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
          monotone_constraints='()', n_estimators=500, n_jobs=16,
          num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1,
          scale_pos_weight=None, subsample=0.75, tree_method='exact',
          validate_parameters=1, verbosity=None)

In [9]:
x_valid, y_valid, _ = prepare_data(validation_data)

In [10]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['id'])])
    
predictions = (x_valid.groupby('srch_id')
               .apply(lambda x: predict(model, x)))

In [11]:
unique_validation_ids = validation_data["srch_id"].unique()
unique_validation_ids

array([233039, 233040, 233041, ..., 332782, 332784, 332785], dtype=int64)

In [12]:
grouped_validation_data = validation_data.groupby('srch_id')

In [13]:
total_score = 0
for id in unique_validation_ids:
    valid_group = grouped_validation_data.get_group(id)
    valid_group["prediction"] = predictions[id]
    valid_group = valid_group.sort_values(by="prediction", ascending=False)
    total_score += np.nan_to_num(calculate_ndcg(valid_group))

total_score / len(unique_validation_ids)

C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


0.3237410746710859

In [14]:
xgb_methods = ['gbtree']
for method in xgb_methods:

    model = xgb.XGBRanker(  
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9,  
    max_depth=7, 
    n_estimators=500, 
    subsample=0.75 
    )

    model.fit(x_train, y_train, group=train_groups, verbose=True)

    predictions = (x_valid.groupby('srch_id')
               .apply(lambda x: predict(model, x)))
    
    total_score = 0
    for id in unique_validation_ids:
        valid_group = grouped_validation_data.get_group(id)
        valid_group["prediction"] = predictions[id]
        valid_group = valid_group.sort_values(by="prediction", ascending=False)
        total_score += np.nan_to_num(calculate_ndcg(valid_group))

    print(method, total_score / len(unique_validation_ids))

C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


gbtree 0.3237410746710859


In [15]:
#Read the test data from the provided file and save it in the variable dataset as a DataFrame object
test_data = pd.read_csv("data/processed_testing_data.csv")
test_data

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,score
0,0,1,2013-02-02 15:27:40,24,216,219,3180,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
1,1,1,2013-02-02 15:27:40,24,216,219,5543,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
2,2,1,2013-02-02 15:27:40,24,216,219,14142,2,3.5,1,...,0,19222,1,10,2,0,1,0,0,0
3,3,1,2013-02-02 15:27:40,24,216,219,22393,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
4,4,1,2013-02-02 15:27:40,24,216,219,24194,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,4959178,332787,2013-05-21 11:06:37,24,216,117,32019,4,3.5,0,...,0,19246,2,7,1,0,1,0,0,0
4959179,4959179,332787,2013-05-21 11:06:37,24,216,117,33959,4,3.0,1,...,0,19246,2,7,1,0,1,0,0,0
4959180,4959180,332787,2013-05-21 11:06:37,24,216,117,35240,4,0.0,0,...,0,19246,2,7,1,0,1,0,0,0
4959181,4959181,332787,2013-05-21 11:06:37,24,216,117,94437,4,0.0,0,...,0,19246,2,7,1,0,1,0,0,0


In [16]:
unique_test_ids = test_data["srch_id"].unique()
unique_test_ids

array([     1,      3,      6, ..., 332785, 332786, 332787], dtype=int64)

In [17]:
grouped_test_data = test_data.groupby('srch_id')

In [18]:
x_test = test_data.drop(["date_time", "score"], axis=1)

In [20]:
test_predictions = (x_test.groupby('srch_id')
               .apply(lambda x: predict(model, x)))

In [21]:
df_prediction = pd.DataFrame(columns=["srch_id", "prop_id"])
path = "XGBR_prediction.csv"

for id in unique_test_ids:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    test_group = grouped_test_data.get_group(id)
    test_group["prediction"] = test_predictions[id]
    test_group = test_group.sort_values(by="prediction", ascending=False)
    df_temp = pd.DataFrame()
    df_temp["srch_id"] = test_group["srch_id"]
    df_temp["prop_id"] = test_group["prop_id"]
    df_prediction = pd.concat([df_prediction, df_temp])

writer = csv.writer(open(path, "w"), lineterminator="\n")
writer.writerow(("srch_id", "prop_id"))
writer.writerows(df_prediction)

C:\Users\Helia\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
df_prediction.to_csv("submission.csv", index=False)

In [ ]:
# model = xgb.XGBRanker(  
#     tree_method='gpu_hist',
#     booster='gbtree',
#     objective='rank:pairwise',
#     random_state=42, 
#     learning_rate=0.1,
#     colsample_bytree=0.9, 
#     eta=0.05, 
#     max_depth=6, 
#     n_estimators=500, 
#     subsample=0.75 
#     )

# for i in range(5):
#     model.fit(x_train, y_train, group=train_groups, verbose=True)

#     predictions = (x_valid.groupby('srch_id')
#             .apply(lambda x: predict(model, x)))
#     total_score = 0

#     for id in unique_validation_ids:
#         valid_group = grouped_validation_data.get_group(id)
#         valid_group["prediction"] = predictions[id]
#         valid_group = valid_group.sort_values(by="prediction", ascending=False)
#         total_score += np.nan_to_num(calculate_ndcg(valid_group))

#     print(i, total_score / len(unique_validation_ids))